# E02: Building blocks of variable interactions

A crucial factor to consider in causal inference is the interaction among variables. Three foundational building blocks serve as the cornerstones of causal interactions: mediator, confounder, and collider effects. These interactions can intertwine and manifest concurrently, leading to more complex causal networks.  Hence, correctly capturing these interactions is key to faithfully characterizing more general causal patterns. These cases pose significant challenges for causal inference methods in the context of time series analysis and represent the fundamentals of interactions between variables. In this notebook, we will show how SURD can be applied to four different scenarios representative of the building blocks of causal interactions.

In [1]:
import os

import numpy as np  # type: ignore
import matplotlib.pyplot as plt  # type: ignore
from surd import surd

from surd.utils import analytic_eqs as cases

np.random.seed(10)

# Configure matplotlib to use LaTeX for text rendering and set font size
plt.rcParams["text.usetex"] = True
plt.rcParams.update({"font.size": 22})

## Problem set-up

We define some important parameters for the calculation. `Nt` represents the number of time steps we will use to integrate the system over time. `samples` represents the number of samples to be used in the analysis after removing transient effects. `nbins` determines the number of states or discretization for the histograms and `nlag` specifies the time lag for performing causal analysis. The number of bins should be selected so that $N_{\rm{samples}} > {N_{\rm{bins}}}^d$, where $d$ represents the dimension of the pdf to be estimated.

In [2]:
# Number of time steps to perform the integration of the system
n_time_steps = 5 * 10**7
# Number of samples to be considered (remove the transients)
samples = n_time_steps - 10000
n_bins = 50  # Number of bins to discretize the histogram
n_lag = 1  # Time lag to perform the causal analysis

## Mediator variable

The first example corresponds to the system $Q_3 \rightarrow Q_2 \rightarrow Q_1$, where $Q_3$ acts on $Q_1$ through the mediator variable $Q_2$. The equations governing the system are as follows:
\begin{align*}
Q_1(n+1) &= \sin[Q_2(n)] + 0.001W_1(n) \\
Q_2(n+1) &= \cos[Q_3(n)] + 0.01W_2(n) \\
Q_3(n+1) &= 0.9Q_3(n) + 0.1W_3(n),
\end{align*}
where $W_i$ represents stochastic forcing on the variable and follows a Gaussian distribution with a mean of zero and a standard deviation of one. The system is initially set to $Q_1(1) = Q_2(1) = Q_3(1) = 0$.

In [ ]:
# Define paths for saving/loading data for each system
formatted_n_time_steps = (
    "{:.0e}".format(n_time_steps).replace("+0", "").replace("+", "")
)
filepath = os.path.join("../data", f"mediator_steps_{formatted_n_time_steps}.npy")

# Check if data is saved and load it, otherwise generate and save
if os.path.isfile(filepath):
    data = np.load(filepath)
    print("Loaded data for mediator")
else:
    qs = cases.mediator(n_time_steps)
    data = np.array([q[-samples:] for q in qs])
    np.save(filepath, data)
    print("Generated and saved data for mediator")

In [ ]:
# Since the calculations of causalities are not coupled between variables, you can run each calculation in parallel and obtain a speed up corresponding to the number of variables
# n_vars = X.shape[0]
# Prepare subplots
# fig, axs = plt.subplots(
#     n_vars, 2, figsize=(9, 2.3 * n_vars), gridspec_kw={"width_ratios": [35, 1]}
# )
# surd.run(X, n_lag, n_bins, axs)

# Uncomment this line and comment the previous line to run this code in series
rd_results, sy_results, mi_results, info_leak_results = surd.run_parallel(data, n_lag, n_bins, plot=False)

In [ ]:
rd_results.keys()

## Confounder variable

The second example considered corresponds to a system where $Q_3$ is a confounder variable to $Q_1$ and $Q_2$, i.e., $Q_3 \rightarrow Q_1$ and $Q_3 \rightarrow Q_2$.

\begin{align*}
    Q_1(n+1) &= \sin[Q_1(n) + Q_3(n)] + 0.01W_1(n)\\
    Q_2(n+1) &= \cos[Q_2(n) - Q_3(n)] + 0.01W_2(n) \\
    Q_3(n+1) &= 0.5Q_3(n) + 0.1W_3(n)
\end{align*}

In [ ]:
# Define paths for saving/loading data for each system
filepath = os.path.join("../data", f"confounder_Nt_{formatted_n_time_steps}.npy")

# Check if data is saved and load it, otherwise generate and save
if os.path.isfile(filepath):
    X = np.load(filepath)
    print("Loaded data for confounder")
else:
    qs = cases.confounder(n_time_steps)
    X = np.array([q[-samples:] for q in qs])
    np.save(filepath, X)
    print("Generated and saved data for confounder")

In [ ]:
n_vars = X.shape[0]

# Prepare Subplots
fig, axs = plt.subplots(
    n_vars, 2, figsize=(9, 2.3 * n_vars), gridspec_kw={"width_ratios": [35, 1]}
)
I_R, I_S, MI, info_leak = surd.run_parallel(X, n_vars, n_lag, n_bins, axs)

plt.tight_layout(w_pad=-8, h_pad=0)
plt.show()

## Synergistic collider

The third example considered here corresponds to the system $[Q_2,Q_3] \rightarrow Q_1$, where $Q_2$ and $Q_3$ work together to influence $Q_1$. In reality, variables $Q_2$ and $Q_3$ behave as a single random variable that drives $Q_1$.

\begin{align*}
    Q_1(n+1) &= \sin[Q_2(n)Q_3(n)] + 0.001W_1(n) \\
    Q_2(n+1) &= 0.5Q_2(n) + 0.1W_2(n) \\
    Q_3(n+1) &= 0.5Q_3(n) + 0.1W_3(n)
\end{align*}

In [ ]:
# Define paths for saving/loading data for each system
filepath = os.path.join("../data", f"synergistic_Nt_{formatted_n_time_steps}.npy")

# Check if data is saved and load it, otherwise generate and save
if os.path.isfile(filepath):
    X = np.load(filepath)
    print("Loaded data for synergistic")
else:
    qs = cases.synergistic_collider(n_time_steps)
    X = np.array([q[-samples:] for q in qs])
    np.save(filepath, X)
    print("Generated and saved data for synergistic")

n_vars = X.shape[0]

# Prepare Subplots
fig, axs = plt.subplots(
    n_vars, 2, figsize=(9, 2.3 * n_vars), gridspec_kw={"width_ratios": [35, 1]}
)

I_R, I_S, MI, info_leak = surd.run_parallel(X, n_vars, n_lag, n_bins, axs)

plt.tight_layout(w_pad=-8, h_pad=0)
plt.show()

## Redundant collider

The last example represents the system $Q_2 \equiv Q_3 \rightarrow Q_1$, where $Q_3$ is the same as $Q_2$. In this situation, both $Q_2$ and $Q_3$ provide the same information about their influence on the future of $Q_1$.

\begin{align*}
    Q_1(n+1) &=  0.3Q_1(n) + \sin[Q_2(n)Q_3(n)] + 0.001 W_1(n)\\
    Q_2(n+1) &= 0.5 Q_2(n) + 0.1W_2(n) \\
    Q_3(n+1) &\equiv Q_2(n+1)
\end{align*}

In [ ]:
# Define paths for saving/loading data for each system
filepath = os.path.join("../data", f"redundant_Nt_{formatted_n_time_steps}.npy")

# Check if data is saved and load it, otherwise generate and save
if os.path.isfile(filepath):
    X = np.load(filepath)
    print("Loaded data for redundant")
else:
    qs = cases.redundant_collider(n_time_steps)
    X = np.array([q[-samples:] for q in qs])
    np.save(filepath, X)
    print("Generated and saved data for redundant")

rd_results, sy_results, mi_results, info_leak_results = surd.run_parallel(X, n_lag, n_bins, plot=True)
